# QBot Process Creation
Detects QBot like process executions

## Rule Content
```
- title: QBot Process Creation
  id: 4fcac6eb-0287-4090-8eea-2602e4c20040
  status: experimental
  description: Detects QBot like process executions
  author: Florian Roth
  date: 2019/10/01
  references:
  - https://twitter.com/killamjr/status/1179034907932315648
  - https://app.any.run/tasks/2e0647b7-eb86-4f72-904b-d2d0ecac07d1/
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection1:
      ParentImage: '*\WinRAR.exe'
      Image: '*\wscript.exe'
    selection2:
      CommandLine: '* /c ping.exe -n 6 127.0.0.1 & type *'
    condition: selection1 or selection2
  fields:
  - CommandLine
  - ParentCommandLine
  falsepositives:
  - Unlikely
  level: critical

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='((process_parent_path.keyword:*\\WinRAR.exe AND process_path.keyword:*\\wscript.exe) OR process_command_line.keyword:*\ \/c\ ping.exe\ \-n\ 6\ 127.0.0.1\ &\ type\ *)')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()